<a href="https://colab.research.google.com/github/nyoons/ESAA/blob/main/Ch_09_%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C_%EA%B3%BC%EC%A0%9C2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**05. 콘텐츠 기반 필터링 실습 - TMDB 5000 영화 데이터 세트**
###**장르 속성을 이용한 영화 콘텐츠 기반 필터링**
콘텐츠 기반 필터링 : 사용자가 특정 영화를 감상하고 그 영화를 좋아했다면 그 영화와 비슷한 특성, 구성 요소 등을 가진 다른 영화를 추천하는 것. 이렇게 상품 간의 유사성을 판단하는 기준이 상품을 구성하는 다양한 콘텐츠(ex. 영화에서 장르, 감독, 배우, 키워드 등)를 기반으로 하는 방식이 바로 콘텐츠 기반 필터링.

콘텐츠 기반 필터링 추천 시스템을 영화를 선택하는데 중요한 요소인 영화 장르 속성을 기반으로 만들어보자. 장르 칼럼 값의 유사도를 비교한 뒤 그중 높은 평점을 가지는 영화를 추천하는 방식이다.

###**데이터 로딩 및 가공**

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

movies=pd.read_csv('tmdb_5000_movies.csv')
print(movies.shape)
movies.head(1)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


4803*20, 영화에 대한 다양한 메타 정보. 이 중 콘텐츠 기반 필터링 추천 분석에 사용할 주요 칼럼만 추출하자.

In [3]:
movies_df=movies[['id', 'title', 'genres', 'vote_average', 'vote_count', 'popularity', 'keywords', 'overview']]

주의할 점 : genres, keywords 등의 칼럼을 보면 파이썬 리스트 내부에 여러 개의 딕셔너리가 있는 형태의 문자열로 표기되어 있다. 한번에 여러 값을 표현하기 위한 표기 방식. **필요한 정보를 추출**할 필요가 있다.

해당 칼럼이 어떤 형태로 되어 있는지 확인해보자.

In [4]:
pd.set_option('max_colwidth', 100)
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


genres 칼럼의 문자열을 분해해 개별 장르를 파이썬 리스트 객체로 추출하자. ast 모듈의 literal_eval() 사용하면 이 문자열을 list[dict1, dict2] 객체로 만들 수 있다.

In [5]:
from ast import literal_eval
movies_df['genres']=movies_df['genres'].apply(literal_eval)
movies_df['keywords']=movies_df['keywords'].apply(literal_eval)

In [7]:
movies_df.head(3)

,id,title,genres,vote_average,vote_count,popularity,keywords,overview
0,19995,Avatar,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {...",7.2,11800,150.437577,"[{'id': 1463, 'name': 'culture clash'}, {'id': 2964, 'name': 'future'}, {'id': 3386, 'name': 'sp...","In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ..."
1,285,Pirates of the Caribbean: At World's End,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 28, 'name': 'Action'}]",6.9,4500,139.082615,"[{'id': 270, 'name': 'ocean'}, {'id': 726, 'name': 'drug abuse'}, {'id': 911, 'name': 'exotic is...","Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of t..."
2,206647,Spectre,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 80, 'name': 'Crime'}]",6.3,4466,107.376788,"[{'id': 470, 'name': 'spy'}, {'id': 818, 'name': 'based on novel'}, {'id': 4289, 'name': 'secret...",A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. Whil...


이제 장르 칼럼은 문자열이 아니라 실제 리스트 내부에 여러 장르 딕셔너리로 구성된 객체를 갖는다. 장르명만 리스트 객체로 추출하자.

In [8]:
movies_df['genres']=movies_df['genres'].apply(lambda x : [y['name'] for y in x])
movies_df['keywords']=movies_df['keywords'].apply(lambda x : [y['name'] for y in x])
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


###장르 콘텐츠 유사도 측정
A의 장르도 여러개, B의 장르도 여러개라면 어떻게 유사도를 측정할까? 가장 간단한 방법은 genres를 문자열로 변경한 뒤, 이를 countvectorizer로 피처 벡터화한 행렬 데이터 값을 코사인 유사도로 비교하는 것.

genres 칼럼을 기반으로 하는 콘텐츠 기반 필터링은 다음 단계로 구현하자.
1. 문자열로 변환된 genres 칼럼을 count 기반으로 피처 벡터화 변환
2. genres 문자열을 피처 벡터화 행렬로 변환한 데이터 세트를 코사인 유사도를 통해 비교. 이를 위해 데이터 세트의 레코드별로 타 레코드와 장르에서 코사인 유사도 값을 가지는 객체를 생성
3. 장르 유사도가 높은 영화 중 평점이 높은 순으로 영화 추천

먼저 genres 칼럼을 문자열로 변환한 뒤 count 벡터화하자. 리스트 객체 내 개별 값을 연속된 문자열로 변환하려면 일반적으로 ('구분문자').join(리스트 객체) 을 사용.

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

#countvectorizer를 적용하기 위해 공백 문자로 word 단위가 구분되는 문자열로 변환
movies_df['genres_literal']=movies_df['genres'].apply(lambda x:(' ').join(x))
count_vect=CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat=count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


count 벡터화 후 4803*276 됨.

이렇게 생성된 피처 벡터 행렬에 사이킷런의 cosine_similarity()를 이용해 코사인 유사도를 계산하자. 코사인 유사도는 기준 행과 비교 행의 코사인 유사도를 행렬 형태로 반환하는 함수.

In [10]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim=cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:1])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]]


genre_sim 객체는 movies_df의 genre_literal 칼럼을 피처 벡터화한 행렬 데이터의 행별 유사도 정보를 갖는다. 결국 movies_df의 행별 장르 유사도 값을 갖고 있는 것. movies_df를 장르 기준으로 콘텐츠 기반 필터링 수행하려면 movies_df의 개별 레코드에 대해 가장 장르 유사도가 높은 순으로 다른 레코드를 추출해야 하는데, 이를 위해 앞에서 생성한 genre_sim을 이용.

genre_sim 객체의 기준 행별로 비교 대상이 되는 행의 유사도 값이 높은 순으로 정렬된 행렬의 위치 인덱스를 추출하면 된다. 값이 높은 순으로 정렬된 비교 대상 행의 유사도 값이 아니라 비교 대상 행의 위치 인덱스임을 주의. argsort()[:, ::-1]을 이용하면 유사도가 높은 순으로 정리된 genre_sim 객체의 비교 행 위치 인덱스 값을 간편하게 얻을 수 있다.

In [11]:
genre_sim_sorted_ind=genre_sim.argsort()[:, ::-1]
print(genre_sim_sorted_ind[:1])

[[   0 3494  813 ... 3038 3037 2401]]


이 결과가 의미하는 것은 0번 레코드의 경우 자신인 0번 레코드를 제외하면 3494번 레코드가 가장 유사도가 높고, 그 다음이 813 레코드..... 가장 유사도가 낮은 레코드는 2401번 레코드 라는 것.

이 위치 인덱스를 이용해 특정 레코드와 코사인 유사도가 높은 다른 레코드를 추출할 수 있다.

###**장르 콘텐츠 필터링을 이용한 영화 추천**
이제 장르 유사도에 따라 영화 추천하는 함수를 만들자.

In [12]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
  #인자로 입력된 movie_df에서 title 칼럼이 입력된 title_name값인 DF 추출
  title_movie=df[df['title']==title_name]

  #title_naemd를 가진 df의 index 객체를 ndarray로 반환하고
  #sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top n개의 index 추출
  title_index=title_movie.index.values
  similar_indexes=sorted_ind[title_index, :(top_n)]

  #추출된 top_n index 출력. top_n index는 2차원 데이터임.
  #df에서 index로 사용하기 위해 1차원 array로 변경해야함
  print(similar_indexes)
  similar_indexes=similar_indexes.reshape(-1)

  return df.iloc[similar_indexes]

이 함수를 이용해 영화 '대부'와 장르별로 유사한 영화 10개를 추천해보자.

In [13]:
similar_movies=find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title', 'vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


낯설거나, 이해하기 어려운 영화, 평점 낮은 영화도 존재. 개선이 필요.

좀 더 많은 후보군을 선정한 뒤 영화의 평점 vote_average에 따라 필터링해서 최종 추천하는 방식으로 변경하자. vote_average를 사용할 때 주의할 점은 평점 높아도 관객 한명이 평가한 것이면 신뢰할 수 없음.

In [14]:
movies_df[['title', 'vote_average', 'vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


이렇게 평가 횟수가 매우 적은 것들. 왜곡된 평점 데이터를 피할 수 있도록 평점에 평가 횟수를 반영할 수 있는 새로운 평가 방식이 필요.

평가 횟수에 가중치가 부여된 평점 방식을 이용하자.

가중 평점=(v/(v+m)) * R + (m/(v+m)) * C
- v : 개별 영화에 평점을 투표한 횟수 = vote_count
- m : 평점을 부여하기 위한 최소 투표 횟수 = 투표 횟수에 따른 가중치 직접 조절
- R : 개별 영화에 대한 평균 평점 = vote_average
- C : 전체 영화에 대한 평균 평점

우리는 m=전체 투표 횟수에서 상위 60%에 해당하는 횟수를 기준으로 정하자.

In [15]:
C=movies_df['vote_average'].mean()
m=movies_df['vote_count'].quantile(0.6)
print('C:', round(C, 3), 'm:', round(m, 3))

C: 6.092 m: 370.2


기존 평점을 새로운 가중 평점으로 변경하는 함수 생성하고 새로운 평점 정보 값을 만들자.

In [20]:
percentile=0.6
m=movies_df['vote_count'].quantile(percentile)
C=movies_df['vote_average'].mean()

def weighted_vote_average(record):
  v=record['vote_count']
  R=record['vote_average']

  return ( (v/(v+m)) * R ) + ( (m/(v+m)) * C )

movies_df['weighted_vote']=movies.apply(weighted_vote_average, axis=1)

새롭게 부여된 weighted_vote 평점을 기준으로 다시 상위 10개 영화를 추출하자.

In [19]:
movies_df.head(4)

,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",7.2,11800,150.437577,"[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa...","In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ...",Action Adventure Fantasy Science Fiction
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]",6.9,4500,139.082615,"[ocean, drug abuse, exotic island, east india trading company, love of one's life, traitor, ship...","Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of t...",Adventure Fantasy Action
2,206647,Spectre,"[Action, Adventure, Crime]",6.3,4466,107.376788,"[spy, based on novel, secret agent, sequel, mi6, british secret service, united kingdom]",A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. Whil...,Action Adventure Crime
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]",7.6,9106,112.312950,"[dc comics, crime fighter, terrorist, secret identity, burglar, hostage drama, time bomb, gotham...","Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's c...",Action Crime Drama Thriller


In [21]:
movies_df[['title', 'vote_average', 'weighted_vote', 'vote_count']].sort_values('weighted_vote', ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


새롭게 정의된 평점 기준에 따라 영화를 추천해보자. 장르 유사성이 높은 영화를 top_n의 2배수만큼 후보군으로 선정하고, weighted_vote 칼럼 값이 높은 순으로 top_n만큼 추출하는 방식으로 find_sim_movie 함수를 변경하자.

In [22]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
  title_movie=df[df['title']==title_name]
  title_index=title_movie.index.values

  #top_n의 2배에 해당하는 장르 유사성이 높은 인덱스 추출
  similar_indexes=sorted_ind[title_index, :(top_n*2)]
  similar_indexes=similar_indexes.reshape(-1)
  
  #기준 영화 인덱스는 제외
  similar_indexes=similar_indexes[similar_indexes != title_index]

  #top_n의 2배에 해당하는 후보군에서 weighted_vote 높은 순으로 top_n 만큼 추출
  return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

similar_movies=find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title', 'vote_average', 'weighted_vote']]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427


이전 추천보다 훨씬 나아진 추천 리스트. 

하지만 장르만으로는 추천하기에 부족하다. 확장 필요.